In [1]:
# Learning notes
# 1. Chromedriver is located in "C:\Windows". When you need to update the driver, replace the .exe file in there.

## Prerequisites

In [1]:
import math
import pandas as pd

import plotly.subplots as sp
import plotly.graph_objects as go

In [2]:
import time
import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup

import os
import glob
pause_time = 3


## Scrape Reviews

In [3]:
# estepalace: 'https://www.google.com/maps/place/Estepalace+Hair+Transplant+Center/@41.0681014,28.9914843,17z/data=!3m1!4b1!4m6!3m5!1s0x14cab820372c2ef3:0x711c752f40eda6ca!8m2!3d41.0681014!4d28.993673!16s%2Fg%2F1tjj8mtd'
# estehair: 'https://www.google.com/maps/place/Estet+Istanbul+Saglik+%26+Estetik/@41.0838758,29.011377,17z/data=!3m1!4b1!4m6!3m5!1s0x14cab66f08308d5b:0x87a8dbfee5c3f4b8!8m2!3d41.0838758!4d29.0135657!16s%2Fg%2F1tctn_f4'

In [4]:
driver = webdriver.Chrome()  # Optional argument, if not specified will search path.

driver.implicitly_wait(pause_time)

url = 'https://www.google.com/maps/place/Estepalace+Hair+Transplant+Center/@41.0681014,28.9914843,17z/data=!3m1!4b1!4m6!3m5!1s0x14cab820372c2ef3:0x711c752f40eda6ca!8m2!3d41.0681014!4d28.993673!16s%2Fg%2F1tjj8mtd'
driver.get(url)
time.sleep(pause_time)

# # Accept the cookies (ctrl q)
# driver.find_element(By.XPATH, '//*[@id="yDmH0d"]/c-wiz/div/div/div/div[2]/div[1]/div[3]/div[1]/div[1]/form[2]/div/div/button/span').click()
# time.sleep(pause_time)



In [5]:
#Find the total number of ratings
total_number_of_ratings = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/div[2]/span[2]/span[1]/span').text.split(" ")[0]
total_number_of_ratings = total_number_of_ratings.replace("(","")
total_number_of_ratings = int(total_number_of_ratings.replace(")",""))
print(total_number_of_ratings)

147


In [6]:
# Click on reviews and lay them out
xpath_str = '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div/div/button[2]/div[2]/div[2]'
driver.find_element(By.XPATH, xpath_str).click()

In [7]:
# # Alternative way to scroll down the review window
# html = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]')
# html.send_keys(Keys.END)

# Get scroll height
# last_height = driver.execute_script("return document.body.scrollHeight")

n_loaded_objects = int(len(driver.find_elements(By.CLASS_NAME, 'jftiEf')))
ele = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]')

# Scroll the page down to the bottom until there are no more reviews to load
while n_loaded_objects < int(total_number_of_ratings):
    driver.execute_script('arguments[0].scrollBy(0, 5000);', ele)
    time.sleep(pause_time)
    n_loaded_objects = int(len(driver.find_elements(By.CLASS_NAME, 'jftiEf')))
    print(n_loaded_objects)



20
30
40
50
60
70
80
90
100
110
120
130
140
147
147


In [8]:
elems = driver.find_elements(By.CLASS_NAME, 'kyuRq.fontTitleSmall.WOKzJe') 
# By clicking on the "See original" buttons, each review is reverted back to its original language
for elem in elems:
        if 'See original' in elem.text:
                try:
                    elem.click()
                except:
                       continue
# <button class="kyuRq fontTitleSmall WOKzJe" role="switch" aria-checked="true" aria-controls="ChZDSUhNMG9nS0VJQ0FnSURRcVpUdkVBEAE" data-review-id="ChZDSUhNMG9nS0VJQ0FnSURRcVpUdkVBEAE" jsaction="pane.review.showReviewInOriginal" jslog="170842; track:click; mutable:true;metadata:WyIwYWhVS0V3aXhxZF9PNWE2QkF4WFdpdjBISGN5ekFzWVE0UjRJTVNnSiJd"> <span><span class="fontBodySmall iUHfzf">Translated by Google</span> <span class="fontBodySmall iUHfzf">・</span></span> <span>See original (Turkish)</span></button>

In [9]:
elems = driver.find_elements(By.CLASS_NAME, 'w8nwRe.kyuRq') 
# By clicking on the "More" buttons, each review that is too long to be fully displayed, is extended to its full format.
for elem in elems:
        if elem.text == "More":
                try:
                    elem.click()
                except:
                       continue


In [11]:
# Find all review elements just once
review_elements = driver.find_elements(By.CLASS_NAME, 'jftiEf')

# Initialize lists to store review data
reviews_data = []

# Loop through each review element to extract information
for review_element in review_elements:
    review_data = {}
    
    # Extract user ID and rating ID
    user_info_element = review_element.find_element(By.CLASS_NAME, 'WEBjve')
    review_data['user_id'] = user_info_element.get_attribute('data-href')
    review_data['rating_id'] = user_info_element.get_attribute('data-review-id')
    
    # Extract rating date and star rating
    rating_date_element = review_element.find_element(By.CLASS_NAME, 'rsqaWe')
    review_data['rating_date'] = rating_date_element.text
    
    star_rating_element = review_element.find_element(By.CLASS_NAME, 'kvMYJc')
    review_data['star_rating'] = star_rating_element.get_attribute('aria-label')
    
    # Attempt to extract review text and review language
    try:
        review_text_element = review_element.find_element(By.CLASS_NAME, 'MyEned')
        review_data['review'] = review_text_element.text
        review_data['review_lang'] = review_text_element.get_attribute('lang')
    except NoSuchElementException:
        review_data['review'] = ''
        review_data['review_lang'] = ''
    
    # Attempt to extract response details if present
    try:
        response_elements = review_element.find_elements(By.CLASS_NAME, 'DZSIDd')
        response_date_element = response_elements[0]  # Assuming first element is the response date
        review_data['response_date'] = response_date_element.text if response_date_element else ''
        
        response_text_elements = review_element.find_elements(By.CLASS_NAME, 'wiI7pd')
        response_text_element = response_text_elements[-1]  # Assuming the last element is the response text
        review_data['response'] = response_text_element.text if response_text_element else ''
        review_data['response_lang'] = response_text_element.get_attribute('lang') if response_text_element else ''
    except IndexError:
        review_data['response_date'] = ''
        review_data['response'] = ''
        review_data['response_lang'] = ''
    
    # Add the collected data to the list
    reviews_data.append(review_data)

# Example: Print out the collected review data
for data in reviews_data:
    print(data)


{'user_id': 'https://www.google.com/maps/contrib/102475968812759470625/reviews?hl=en', 'rating_id': 'ChdDSUhNMG9nS0VJQ0FnSURaamZUdG9nRRAB', 'rating_date': '5 months ago', 'star_rating': '5 stars', 'review': 'Amazing! This place was recommended by my brother who used their service 6 years ago and had great results!\nLast week I was in the clinic and I can’t say enough about how great and professional the whole team was.. literally they become my “Turkey” family.. Aicha came on Sunday ( holiday) to show me the steps of how to wash my hair since I didn’t have enough time to stay more days, Mohamed went above and beyond explaining each and every steps and answer all my questions.. the process of the hair transplant was very relaxing and pain free..Ahmed was always keeping me updated over the phone before and after the surgery, all drivers were friendly and helpful and took care of my luggage’s and didn’t let me even help them… definitely sending more clients… keep up the great work and can

In [18]:
df = pd.DataFrame(reviews_data)

In [19]:
rating_date_tf = []

# Get today's date
today = datetime.date.today()


for raw_rating in df.rating_date:
    units = int(1 if raw_rating.split(' ')[0] == 'a' else raw_rating.split(' ')[0])

    if 'year' in raw_rating:
        factor = 365
    elif 'month' in raw_rating:
        factor = 30
    elif 'week' in raw_rating:
        factor = 7
    elif 'day' in raw_rating:
        factor = 1
    
    approx_days = factor*units
    approx_date = today - datetime.timedelta(days=approx_days)
    rating_date_tf.append(approx_date)

df['rating_date_approx'] = rating_date_tf
df['rating_date_approx'] = pd.to_datetime(df['rating_date_approx'])

In [20]:
response_date_tf = []

# Get today's date
today = datetime.date.today()


for raw_rating in df.response_date:
    try:
        units = int(1 if raw_rating.split(' ')[0] == 'a' else raw_rating.split(' ')[0])

        if 'year' in raw_rating:
            factor = 365
        elif 'month' in raw_rating:
            factor = 30
        elif 'week' in raw_rating:
            factor = 7
        elif 'day' in raw_rating:
            factor = 1
        
        approx_days = factor*units
        approx_date = today - datetime.timedelta(days=approx_days)
        response_date_tf.append(approx_date)
    except:
        response_date_tf.append('')

df['response_date_approx'] = response_date_tf
df['response_date_approx'] = pd.to_datetime(df['response_date_approx'])


In [21]:
df['star_rating'] = [int(x[:1]) for x in df['star_rating']]

In [22]:
df['review'] = [x.replace('\n', ' ') for x in df['review']]
df['response'] = [x.replace('\n', ' ') for x in df['response']]

In [23]:
# df = pd.read_pickle('estepalace.pkl')
df.to_pickle('estepalace.pkl')

In [3]:
df = pd.read_pickle('estepalace.pkl')

In [4]:
df

,user_id,rating_id,rating_date,star_rating,review,review_lang,response_date,response,response_lang,rating_date_approx,response_date_approx
0,https://www.google.com/maps/contrib/1024759688...,ChdDSUhNMG9nS0VJQ0FnSURaamZUdG9nRRAB,9 months ago,5,Amazing! This place was recommended by my brot...,en,,,,2023-11-04,NaT
1,https://www.google.com/maps/contrib/1089830126...,ChZDSUhNMG9nS0VJQ0FnSUNlbzhDMkZBEAE,a year ago,5,"Sehr empfehlenswert, sehr professionell unt to...",de,a year ago,Hakkı bey yorumunuz için çok teşekkür ederiz🙏 ...,tr,2023-08-01,2023-08-01
2,https://www.google.com/maps/contrib/1107359297...,ChdDSUhNMG9nS0VJQ0FnSURab2V6QzJ3RRAB,9 months ago,5,First class care and professionalism. I highly...,en,,,,2023-11-04,NaT
3,https://www.google.com/maps/contrib/1176537476...,ChZDSUhNMG9nS0VJQ0FnSUQyN0tqcUVnEAE,2 years ago,5,I did my hair transplant last week over two da...,en,2 years ago,Hi! Thank you for your comment and rating. You...,en,2022-08-01,2022-08-01
4,https://www.google.com/maps/contrib/1003569451...,ChZDSUhNMG9nS0VJQ0FnSURXbmU2N0hnEAE,2 years ago,5,Hallo! Ich was vor knapp zwei Wochen und habe ...,de,,,,2022-08-01,NaT
...,...,...,...,...,...,...,...,...,...,...,...
150,https://www.google.com/maps/contrib/1155109795...,ChdDSUhNMG9nS0VJQ0FnSURJaTcyWnlnRRAB,5 years ago,5,,,,,,2019-08-02,NaT
151,https://www.google.com/maps/contrib/1166229120...,ChZDSUhNMG9nS0VJQ0FnSUNBNE5TRVVREAE,5 years ago,5,,,,,,2019-08-02,NaT
152,https://www.google.com/maps/contrib/1015640358...,ChZDSUhNMG9nS0VJQ0FnSURBbU92aFVREAE,6 years ago,5,,,,,,2018-08-02,NaT
153,https://www.google.com/maps/contrib/1142814319...,ChdDSUhNMG9nS0VJQ0FnSURRd3NDYTBnRRAB,6 years ago,5,,,,,,2018-08-02,NaT


## Data Analysis

- Avergae rating, review count
- 

In [ ]:
# review_sentiment = []
# review_sentiment_score = []

# response_sentiment = []
# response_sentiment_score = []

# for i in range(len(df)):
#     if df.loc[i, 'review_lang'] == 'en':
#         sa = sa_en(df.loc[i, 'review'])
#         label = sa[0]['label']
#         score = sa[0]['score']

#     elif df.loc[i, 'review_lang'] == 'tr':
#         sa = sa_tr(df.loc[i, 'review'])
#         label = sa[0]['label']
#         score = sa[0]['score']

#     else:
#         label = ''
#         score = ''

#     review_sentiment.append(label)
#     review_sentiment_score.append(score)

# for i in range(len(df)):
#     if df.loc[i, 'response_lang'] == 'en':
#         sa = sa_en(df.loc[i, 'review'])
#         label = sa[0]['label']
#         score = sa[0]['score']

#     elif df.loc[i, 'response_lang'] == 'tr':
#         sa = sa_tr(df.loc[i, 'review'])
#         label = sa[0]['label']
#         score = sa[0]['score']

#     else:
#         label = ''
#         score = ''
        
#     response_sentiment.append(label)
#     response_sentiment_score.append(score)
    

In [ ]:
# df['review_sentiment'] = review_sentiment
# df['review_sentiment_score'] = review_sentiment_score

# df['response_sentiment'] = response_sentiment
# df['response_sentiment_score'] = response_sentiment_score

In [ ]:
df = df.sort_values(by='rating_date_approx', ascending=True).reset_index(drop=True)

In [ ]:
df['star_rating_running_avg'] = [math.ceil(x * 10) / 10 for x in df['star_rating'].expanding().mean()]


In [ ]:
df

In [ ]:
# Group the data by month and count the occurrences of each unique value in 'star_rating' within each month
df_monthly = df.groupby([df['rating_date_approx'].dt.to_period('M'), 'star_rating']).size().unstack()

# Set the index as the months
df_monthly.index = df_monthly.index.to_timestamp()

# Create a line plot for each unique value in 'star_rating'
fig = go.Figure()
for column in df_monthly.columns:
    fig.add_trace(go.Scatter(x=df_monthly.index, y=df_monthly[column], mode='lines', name=str(column)))

# Add title and axis labels
fig.update_layout(title='Count of Star Ratings Over Time',
                  xaxis_title='Month',
                  yaxis_title='Count')

# Show the plot
fig.show()

In [ ]:
# Assuming 'df' is your DataFrame with a 'rating_date_approx' column
# Convert 'rating_date_approx' column to pandas datetime format
df['rating_date_approx'] = pd.to_datetime(df['rating_date_approx'])

# Group the data by month and count the occurrences of ratings within each month
df_monthly = df.groupby(df['rating_date_approx'].dt.to_period('M')).size()

# Create a line plot
fig = go.Figure(data=go.Scatter(x=df_monthly.index.to_timestamp(), y=df_monthly, mode='lines'))

# Add title and axis labels
fig.update_layout(title='Number of Ratings Received Every Month',
                  xaxis_title='Month',
                  yaxis_title='Number of Ratings')

# Show the plot
fig.show()

In [ ]:
# Group the data by month and count the occurrences of each unique value in 'star_rating' within each month
df_monthly_count = df.groupby([df['rating_date_approx'].dt.to_period('M'), 'star_rating']).size().unstack()

# Set the index as the months
df_monthly_count.index = df_monthly_count.index.to_timestamp()

# Group the data by month and calculate the average of 'star_rating' within each month
df_monthly_avg = df.groupby(df['rating_date_approx'].dt.to_period('M')).mean()

# Set the index as the months
df_monthly_avg.index = df_monthly_avg.index.to_timestamp()

# Create subplot figure
fig = sp.make_subplots(rows=2, cols=1, subplot_titles=('Count of Star Ratings', 'Average Star Rating'))

# Add traces for count of star ratings
for column in df_monthly_count.columns:
    fig.add_trace(go.Scatter(x=df_monthly_count.index, y=df_monthly_count[column], mode='lines', name=str(column)), row=1, col=1)

# Add traces for average star rating
fig.add_trace(go.Scatter(x=df_monthly_avg.index, y=df_monthly_avg['star_rating'], mode='lines', name='Average'), row=2, col=1)

# Update layout
fig.update_layout(title='Star Rating Analysis',
                  height=600,
                  showlegend=False)

# Set axis labels for each subplot
fig.update_yaxes(title_text='Count', row=1, col=1)
fig.update_yaxes(title_text='Average Rating', row=2, col=1)
fig.update_xaxes(title_text='Month', row=2, col=1)

# Add legend
fig.update_layout(showlegend=True)

# Show the plot
fig.show()

In [ ]:
# Group the data by month and count the total number of star ratings received per month
df_monthly_total = df.groupby(df['rating_date_approx'].dt.to_period('M')).size()

# Group the data by month and star rating, and count the number of star ratings received per month for each rating
df_monthly_total_grouped = df.groupby([df['rating_date_approx'].dt.to_period('M'), 'star_rating']).size().unstack()

# Group the DataFrame by month and select the last observation in each month
df_starrating_running_avg = df.groupby(df['rating_date_approx'].dt.to_period('M'))['star_rating_running_avg'].last()

# Set the index as the months
df_monthly_total.index = df_monthly_total.index.to_timestamp()
df_monthly_total_grouped.index = df_monthly_total_grouped.index.to_timestamp()
df_starrating_running_avg.index = df_starrating_running_avg.index.to_timestamp()


# Create the subplot figure
fig = sp.make_subplots(rows=2, cols=1, subplot_titles=('Number of Star Ratings', 'Running Average Star Rating'))

# # Add traces for total number of star ratings received per month
# fig.add_trace(go.Scatter(x=df_monthly_total.index, y=df_monthly_total, mode='lines', name='Total Count'), row=1, col=1)

# Add traces for number of star ratings received per month grouped by rating
for column in df_monthly_total_grouped.columns:
    fig.add_trace(go.Scatter(x=df_monthly_total_grouped.index, y=df_monthly_total_grouped[column], mode='lines', name=str(column)), row=1, col=1)

# Add trace for running average star rating
fig.add_trace(go.Scatter(x=df_monthly_total_grouped.index, y=df_starrating_running_avg, mode='lines', name='Running Average'), row=2, col=1)

# Update layout
fig.update_layout(title='Star Rating Analysis',
                  height=600,
                  showlegend=False)

# Set axis labels for each subplot
fig.update_yaxes(title_text='Count', row=1, col=1)
fig.update_yaxes(title_text='Running Average', row=2, col=1)
fig.update_xaxes(title_text='Month', row=2, col=1)

# Add legend
fig.update_layout(showlegend=True)

# Show the plot
fig.show()